In [2]:
import pandas as pd
import numpy as np
import os
import sys
import pandas as pd
sys.path.append('..')
from tools import save_pickle

### Search in real disk

In [3]:
def case2path(x_p):
    cur_dirs = os.listdir(x_p)
    result = {}
    for dir_n in cur_dirs:
        case = dir_n[:12]
        if case in result:
            continue
        result[case] = os.path.join(x_p, dir_n)
    return pd.DataFrame.from_dict(result, orient='index', columns=['path'])

def merge_table_creator(selected_p, target_p='../data/Target.xlsx'):
    target = pd.read_excel(target_p)
    case_path_df = case2path(selected_p)
    merge_table = case_path_df.merge(target, left_index=True, right_on='sample')
    print(len(merge_table)/len(target))
    merge_table.reset_index(drop=True, inplace=True)
    return merge_table

In [4]:
merge_table = merge_table_creator('../../selected/')

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../selected/'

In [ ]:
merge_table.head()

In [ ]:
print(f'{len(merge_table)} cases are used')

Use space to save time. I saved the image paths for each case in a dict.

In [ ]:
def saving(merge_table):
    img_disk = {}
    img_count = {}
    for item in merge_table.iterrows():
        path = item[1][0]
        case = item[1][1]
        pool = os.listdir(path)
        img_disk[case] = tuple(os.path.abspath(os.path.join(path, f)) for f in pool)
        img_count[case] = len(pool)
    save_pickle(img_disk, '../../archive/', name='img_disk')
    return img_count

In [ ]:
img_count = saving(merge_table)

Check the minimum of img number

In [ ]:
from math import inf
def check_min(img_count):
    ceil = inf
    record = None
    for k, v in img_count.items():
        if v < ceil:
            ceil = v
            record = k
    print(f'{record} has the minimum number of imgs {ceil}')

In [ ]:
check_min(img_count)

### Compare with raw slides

In [ ]:
pixel_to_micron = 2.43 # 10X power

Next, combine slides metainfo and the outcome of Nesnet selection.

In [ ]:
img_size = pd.read_csv('../data/power.txt', sep=r'\t', engine='python')
img_size['Name'] = img_size['Name'].str[:12]
img_size['Size'] = img_size['Size'].str.split(',')
img_sel = pd.DataFrame.from_dict(img_count, orient='index')
img_sel.columns = ['num']
img_comp = img_sel.merge(img_size, left_index=True, 
                         right_on='Name',how='inner').drop_duplicates('Name')

In [ ]:
img_comp.head()

Then, get useful information from raw data.

In [ ]:
tile_area = 96 * 96
p_to_area = pixel_to_micron ** 2

img_comp['area_10X'] = img_comp['Size'].apply(
    lambda x:int(x[0].replace("'", ""))*int(x[1].replace("'", ""))
)/((img_comp['Power']/10)**2)

img_comp['ratio'] = tile_area * img_comp['num'] / img_comp['area_10X']
img_comp['ori_real_area'] = img_comp['area_10X'] * p_to_area
img_comp['sel_real_area'] = img_comp['ratio'] * img_comp['ori_real_area']

The case with the most selected image is TCGA-D8-A1XR. It's also the one with the largest seletion ratio.

In [ ]:
img_comp.loc[img_comp['num'] == img_comp['num'].max()]

In [ ]:
img_comp.loc[img_comp['ratio'] == img_comp['ratio'].max()]

In [ ]:
img_comp.loc[img_comp['ratio'] == img_comp['ratio'].min()]

In [ ]:
img_comp.loc[img_comp['num'] == img_comp['num'].min()]

I have clipped the minimal number to 50 to better train the model. So lots of cases are with 50 imgs selected.

### Summary

In [ ]:
img_comp.describe()